# 音声データについて
今回使用するのは、[ATRデジタル音声データベース](https://www.atr-p.com/products/sdb.html#DIGI) セットBです。これは音素バランス文のデータベースで、音声認識研究の初期によく使われていました。

この中から、話者MHTの発声した503文を使います。503文はAセットからJセットまであり、過去の研究にならって、今回はAセットからIセットの450文を訓練データに、Jセットの53文をテストデータに用います。

次はAセット第1文の音声です。

In [0]:
import IPython.display
IPython.display.Audio(filename="MHTSDA01.wav")

次はJセット第1文の音声です。

In [0]:
IPython.display.Audio(filename="MHTSDJ01.wav")

ATRのデータベースには、それぞれの音声のどこからどこまでが何の音素なのかを人手でラベリングした情報が付いています。以下はAセット第1文の音声のラベルです。(単位は 100 ns)
```
0 2450000 sil
2450000 3300000 a
3300000 3550000 r
3550000 4300000 a
4300000 5200000 y
5200000 5500000 u
5500000 5800000 r
5800000 6750000 u
6750000 7200000 g
7200000 8050000 e
8050000 8900000 N
8900000 9650000 j
9650000 10150000 i
10150000 11500000 ts
11500000 12250000 u
12250000 14200000 o
14200000 18600000 pau
18600000 19750000 s
19750000 20150000 u
20150000 20500000 b
20500000 21350000 e
21350000 21850000 t
21850000 22650000 e
22650000 23750000 j
23750000 24200000 i
24200000 24650000 b
24650000 25350000 u
25350000 26200000 N
26200000 26650000 n
26650000 27150000 o
27150000 28150000 h
28150000 29050000 o
29050000 29750000 o
29750000 31000000 e
31000000 31550000 n
31550000 32400000 e
32400000 33250000 j
33250000 33550000 i
33550000 34100000 m
34100000 34750000 a
34750000 35200000 g
35200000 36000000 e
36000000 36600000 t
36600000 37200000 a
37200000 37600000 n
37600000 38350000 o
38350000 38750000 d
38750000 39750000 a
39750000 42150000 sil
```
`sil` は発話前後の無音、`pau` は発話内のポーズ、`N` は撥音、`Q` は促音、`A`, `I` ,`U` ,`E` ,`O` は無声化した母音です。

# 音声の特徴量について
音声認識をする時には波形の情報はあまり役立たないので、まず波形から音声の特徴をよく表す特徴を抽出するのが一般的です。代表的な特徴は振幅スペクトルです。FFTから得たスペクトルは次元が大きすぎて扱いづらいので、かつてはスペクトルをよく表す特徴にさらに変換したものを使っていました。音声認識でよく使われていた特徴量はMFCC(メル周波数ケプストラム係数)です。今回は、パワーと12次のMFCCの合計13次元の特徴、およびそれらの傾き(差分)の、計26次元の特徴を使います。特徴量は、音声に10msずつずらしながら窓をかけて切り出した波形から求めます。したがって、1秒あたりおおよそ100個の特徴ベクトルが得られます。この特徴ベクトル1つに対応する区間をフレームと呼びます。各フレームの特徴量列を入力とし、音素列を出力するようなニューラルネットを学習します。

なお、最近ではMFCCのような特徴は使わず、振幅スペクトルを直接入力することも一般的です。

#  音声データの構造
訓練用データ、およびテスト用データは既に作成済です。

In [0]:
import numpy as np

train = np.load("MHT-train.npy")
test = np.load("MHT-test.npy")

`train` は、450発話分のデータを1つにまとめたもので、このようにデータを束ねたものを**バッチ**と呼びます。各発話は `X`、 `t`、 `lab` の組(タプル)からなります。`X` は各フレームの特徴量を表す $L\times F$ 行列、`t` は各フレームの音素番号からなる $L$ 次元ベクトル、`lab` はその発話の音素番号からなる不定長ベクトル。ただし $L$ はフレーム数、$F$ は特徴量の次元数。

In [0]:
(X, t, lab) = train[0]
X.shape

(422, 26)

上の例は、訓練データの最初の発話(Aセット第1文)が422個の26次元ベクトルからなることを示しています。(4.215秒の音声なので納得)

In [0]:
(X, t, lab) = test[0]
X.shape

(381, 26)

同様に、上の例はテストデータの最初の発話(Jセット第1文)が381個の26次元ベクトルからなることを示しています。

/a/, /i/ などの音素は全部で40種類あり、それぞれに整数に割り当てています。音素リストは `phones` というファイルです。

In [0]:
from sklearn.preprocessing import LabelEncoder

with open('phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
le.fit(phones)
le.classes_

array(['A', 'I', 'N', 'Q', 'U', '_', 'a', 'b', 'by', 'ch', 'd', 'dy', 'e',
       'f', 'g', 'gy', 'h', 'hy', 'i', 'j', 'k', 'ky', 'm', 'my', 'n',
       'ny', 'o', 'p', 'pau', 'py', 'r', 'ry', 's', 'sh', 'sil', 't',
       'ts', 'u', 'w', 'y', 'z'], dtype='<U3')

この中には `_` (ブランク) という特別な記号が含まれており、これはCTCで使うことになります。

Scikit-learn に含まれる LabelEncoder という機能を使うと、音素から整数へ変換できます。例えば音素 /a/ の番号は6です。

In [0]:
le.transform(['a'])

array([6])

`t` は各フレームがどの音素区間の一部かを表しています。次は訓練データの最初の発話についての `t` です。

In [0]:
(X, t, lab) = train[0]
t

array([34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
       34, 34, 34, 34, 34, 34, 34,  6,  6,  6,  6,  6,  6,  6,  6,  6, 30,
       30,  6,  6,  6,  6,  6,  6,  6,  6, 39, 39, 39, 39, 39, 39, 39, 39,
       39, 37, 37, 37, 30, 30, 30, 37, 37, 37, 37, 37, 37, 37, 37, 37, 14,
       14, 14, 14, 14, 12, 12, 12, 12, 12, 12, 12, 12,  2,  2,  2,  2,  2,
        2,  2,  2,  2, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 36,
       36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37,
       37, 37, 37, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
       26, 26, 26, 26, 26, 26, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 32,
       32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 37, 37, 37, 37,  7,  7,  7,
        7, 12, 12, 12, 12, 12, 12, 12, 12, 35, 35, 35, 35, 35, 12, 12, 12,
       12, 12, 12, 12, 12

最初と最後の34は、この区間が無音 sil であることを示しています。